In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] =os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] =os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [29]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(model = "llama3:latest")

embeddings = (
    OllamaEmbeddings(model = "llama3")
)

In [30]:
model.invoke([
    HumanMessage(content="Hi, my name is Smayan and I'm a chief AI engineer.")
])

AIMessage(content="Nice to meet you, Smayan! As a chief AI engineer, that's impressive! What kind of AI projects have you been working on? Are you more focused on natural language processing, computer vision, or perhaps reinforcement learning? Or maybe you're exploring some new and innovative areas in the field of artificial intelligence? I'm excited to hear about your work!", additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2025-01-29T11:47:44.758795757Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1017351480, 'load_duration': 9366005, 'prompt_eval_count': 25, 'prompt_eval_duration': 4000000, 'eval_count': 73, 'eval_duration': 1003000000}, id='run-0f86fb66-887b-4c6c-b652-a5dd05b520d5-0')

In [31]:
model.invoke([
    HumanMessage(content="Hi, my name is Smayan and I'm a chief AI engineer."),
    AIMessage(content="Nice to meet you, Smayan! Congratulations on being a chief AI engineer - that's an impressive title! What kind of projects have you been working on lately? Are there any particularly exciting or challenging AI-related initiatives on your plate right now?"),
    HumanMessage(content="Hey, What is my name and what do i do?")
])

AIMessage(content='According to our initial conversation, your name is Smayan, and you are a chief AI engineer!', additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2025-01-29T11:47:45.088032258Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 321654537, 'load_duration': 9012722, 'prompt_eval_count': 96, 'prompt_eval_duration': 5000000, 'eval_count': 21, 'eval_duration': 306000000}, id='run-f90a2695-0916-417c-877b-064dd39ab4b0-0')

### Message History

In [32]:
from langchain_community.chat_message_histories import ChatMessageHistory
from  langchain_core.chat_history import BaseChatMessageHistory
from  langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [33]:
config={"configurable":{"session_id":"chat1"}}

In [34]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Smayan and I am a Chief AI Engineer")],
    config=config
)

In [35]:
response

AIMessage(content="Nice to meet you, Smayan! As a Chief AI Engineer, you must be working on some exciting projects. Can you tell me more about your role and the types of AI applications you're involved with? Are you focused on developing intelligent systems for industries like healthcare, finance, or customer service?", additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2025-01-29T11:47:45.998160894Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 892289868, 'load_duration': 9471867, 'prompt_eval_count': 24, 'prompt_eval_duration': 4000000, 'eval_count': 61, 'eval_duration': 877000000}, id='run-7cb4b8d5-02c5-4269-be20-7448683f0e90-0')

In [36]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="I remember! Your name is Smayan, and you're a Chief AI Engineer!", additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2025-01-29T11:47:46.283123728Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 272237023, 'load_duration': 8851256, 'prompt_eval_count': 99, 'prompt_eval_duration': 5000000, 'eval_count': 18, 'eval_duration': 257000000}, id='run-884c6da7-a980-4786-92eb-c43f486091d3-0')

In [37]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ('system' , "You are a helpful assistant, answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [38]:
chain.invoke({"messages" : [HumanMessage(content="Hi, my name is Smayan")]})

AIMessage(content="Nice to meet you, Smayan! It's great that you're starting a conversation. Is there something on your mind that you'd like to chat about, or are you looking for some assistance with something specific? I'm here to help in any way I can!", additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2025-01-29T11:47:47.095667991Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 799871293, 'load_duration': 9347675, 'prompt_eval_count': 38, 'prompt_eval_duration': 4000000, 'eval_count': 55, 'eval_duration': 785000000}, id='run-b7abbbac-e519-45fc-910f-1981039759b0-0')

In [39]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [40]:
config = {"configurable" : {"session_id" : "chat2"}}

In [41]:
response = with_message_history.invoke({"messages" : [HumanMessage(content="Hi, my name is Smayan")]}
                                       ,
                                       config=config)

In [42]:
response.content

"Nice to meet you, Smayan! It's great to have you here. How can I assist you today? Do you have any questions or topics you'd like to discuss?"

In [43]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [44]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Smayan")],"language":"Hindi"})
response.content

'नमस्ते Smayan जी! मैं आपका मददगार असिस्टेंट हूँ, आपके सभी प्रश्नों का उत्तर देने के लिए तैयार हूँ. क्या आप से कुछ पूछना चाहते हैं?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [45]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [46]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

"Namaste Krish! मैं आपकी सहायता करने के लिए यहाँ हूँ। आप क्या पूछना चाहते है? (Hello Krish! I'm here to help you. What do you want to ask?)"

In [47]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [48]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [49]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I'm just an assistant, I don't have any information about your personal preferences. But if you want to tell me what kind of ice cream you like, I'd be happy to chat with you about it!"

In [50]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me "whats 2 + 2"!'

In [51]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [52]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't think you told me your name, did you?"

In [53]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask any math problems. You asked about your name!"